<a href="https://colab.research.google.com/github/samsoe/mpg_notebooks/blob/master/Bird_point_count_monitoring_Wrangle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Documentation: [Readme bird point count monitoring](https://drive.google.com/open?id=1PRryJzGOUtfr-fKXzb3tbr48xiaTuAMVk18XFXlvxcM)

# Tools

In [1]:
library(tidyverse)
library(lubridate)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.0     ✔ purrr   0.3.4
✔ tibble  3.0.1     ✔ dplyr   0.8.5
✔ tidyr   1.0.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘lubridate’


The following objects are masked from ‘package:dplyr’:

    intersect, setdiff, union


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




In [2]:
# utilitize replace_with_na function
install.packages("naniar")
library(naniar)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘gridExtra’, ‘visdat’, ‘viridis’, ‘UpSetR’




# Source

In [0]:
# 2020-05-15_bird_surveyInfo_records_function.csv
src = "https://drive.google.com/uc?id=1Cmo8e2H0Td2pWXrB4-_YHwdmUJP2Isnj"

In [0]:
df <- read.csv(src, stringsAsFactors = FALSE)

# Rename Columns

Update column values to [DB Schema](https://docs.google.com/document/d/1PRryJzGOUtfr-fKXzb3tbr48xiaTuAMVk18XFXlvxcM/edit#heading=h.5inr0hjz4m4m) 

In [0]:
new_names <- c('survey_ID', 'survey_date', 'survey_time_start_MDT', 'survey_time_end_MDT', 'survey_visit', 
              'survey_grid_point', 'survey_observer', 'survey_wind_code', 'survey_sky_code',
              'survey_noise_code', 'survey_temperature_F', 'records_ID', 'records_spe_code', 
              'records_abundance', 'records_sex', 'records_detect_song', 'records_detect_call', 
              'records_detect_visual', 'records_detect_other', 'records_detect_distance_min_meters', 
              'records_interval', 'records_location_code', 'habitat_location_description', 
              'function_spe_name_common', 'function_habitat_preference', 'function_food_preference', 
              'function_nesting_behavior', 'function_feeding_behavior', 
              'function_taxa_order', 'function_taxa_family')

df <- setNames(df, new_names)

# Clean

In [6]:
str(df)

'data.frame':	117911 obs. of  30 variables:
 $ survey_ID                         : int  1 1 1 1 1 1 1 1 1 1 ...
 $ survey_date                       : chr  "2010-05-31" "2010-05-31" "2010-05-31" "2010-05-31" ...
 $ survey_time_start_MDT             : int  709 709 709 709 709 709 709 709 709 709 ...
 $ survey_time_end_MDT               : int  719 719 719 719 719 719 719 719 719 719 ...
 $ survey_visit                      : int  1 1 1 1 1 1 1 1 1 1 ...
 $ survey_grid_point                 : int  476 476 476 476 476 476 476 476 476 476 ...
 $ survey_observer                   : chr  "Eric Rasmussen" "Eric Rasmussen" "Eric Rasmussen" "Eric Rasmussen" ...
 $ survey_wind_code                  : int  1 1 1 1 1 1 1 1 1 1 ...
 $ survey_sky_code                   : int  2 2 2 2 2 2 2 2 2 2 ...
 $ survey_noise_code                 : int  2 2 2 2 2 2 2 2 2 2 ...
 $ survey_temperature_F              : int  50 50 50 50 50 50 50 50 50 50 ...
 $ records_ID                        : int  1 2 3 4 5 6 7 

## records_sex

In variable `sex`, limit data to c("Female", "Male", "Unknown"). Correct typo and capitalization errors.

In [7]:
# Show unique values
fct_count(df$records_sex)

f,n
<fct>,<int>
,1
1,2
Female,1869
M,1
male,5
Male,80968
MaMe,1
unknown,5
Unknown,35059


In [8]:
# Display deviants
df %>%
  select(survey_date, records_sex) %>%
  filter(records_sex != "Female" & 
         records_sex != "Male" & 
         records_sex != "Unknown")

survey_date,records_sex
<chr>,<chr>
2010-06-25,unknown
2011-07-01,unknown
2012-07-11,male
2013-06-17,unknown
2013-06-18,unknown
2013-05-22,male
2013-05-17,male
2014-05-16,male
2014-06-14,MaMe


In [9]:
# Capitalize all values
df <- df %>%
  mutate(records_sex = str_to_title(records_sex))

fct_count(df$records_sex)

f,n
<fct>,<int>
,1
1,2
Female,1869
M,1
Male,80973
Mame,1
Unknown,35064


In [0]:
df <- df %>%
  mutate(records_sex = ifelse(records_sex == "M", "Male",
               ifelse(records_sex == "Mame", "Male",
               ifelse(records_sex == 1, "Unknown",
               ifelse(records_sex == "", "Unknown", records_sex)))))

In [11]:
fct_count(df$records_sex)

f,n
<fct>,<int>
Female,1869
Male,80975
Unknown,35067


## records_location_code

In variable `location_code`, capitalize instances of "f" and "u".

In [12]:
fct_count(df$records_location_code)

f,n
<fct>,<int>
0,11959
1,18433
10,141
11,1452
12,860
13,783
14,256
15,5448
16,8384


In [0]:
# Capitalize "f","u"
df <- df %>%
  mutate(records_location_code = str_to_title(records_location_code))

In [14]:
fct_count(df$records_location_code)

f,n
<fct>,<int>
0,11959
1,18433
10,141
11,1452
12,860
13,783
14,256
15,5448
16,8384


## survey_observer

In [15]:
unique(sort(df$survey_observer))

[1] "Amy Seaman"      "Barb Pitman"     "Deb Goslin"      "Debbie Leick"   
[5] "Eric Rasmussen"  "John Csoka"      "Katharine Stone" "n"

In [16]:
df <- df %>%
  mutate(survey_observer = ifelse(survey_observer == "n", 
                                         "unknown", 
                                         survey_observer))

unique(df$survey_observer)

[1] "Eric Rasmussen"  "Katharine Stone" "Debbie Leick"    "unknown"        
[5] "Amy Seaman"      "John Csoka"      "Barb Pitman"     "Deb Goslin"

## records_detect_other


Leave values as is for the forseeable future 

In [17]:
fct_count(df$records_detect_other)

f,n
<fct>,<int>
,117196
125,1
36,1
64,1
70,1
80,1
air sac pulse,1
Boom,7
Boom & Wing noise,1


## records_interval

In [18]:
fct_count(df$records_interval)

f,n
<fct>,<int>
,281
1,85175
1`,1
2,29592
After survey,1838
Before survey,1023
U,1


In [19]:
df <- df %>%
  mutate(records_interval = ifelse(records_interval == '1`', '1', 
                            ifelse(records_interval == '', NA, records_interval)))

fct_count(df$records_interval)

f,n
<fct>,<int>
1,85176
2,29592
After survey,1838
Before survey,1023
U,1
NA,281


## resolve "na" and ""

['Habitat', 'Food', 'Nesting', 'Behavior', 'Order', 'Family'] all have 2 "na" values and 451 "" values.  Update these cells to NA

In [20]:
fct_count(df$function_habitat_preference)

f,n
<fct>,<int>
,451
Forest,19233
Grassland,41108
Lake/Pond,2380
Marsh,1344
Mountains,1353
na,2
Open Woodland,42342
River/Stream,201


In [21]:
df <- df %>%
  mutate_if(is.character, list(~ na_if(., "na"))) %>%
  # would be nice to mutate "na" and "" in a single line
  mutate_if(is.character, list(~ na_if(., ""))) %>%
  # remove "na" and "" factor levels
  droplevels()

fct_count(df$function_habitat_preference)

f,n
<fct>,<int>
Forest,19233
Grassland,41108
Lake/Pond,2380
Marsh,1344
Mountains,1353
Open Woodland,42342
River/Stream,201
Scrub,6067
Shoreline,506


## time

### survey_time_start

In [22]:
sort(unique(df$survey_time_start_MDT))

[1]    0    1   50   98  540  543  544  545  546  547  548  550  551  552  553
 [16]  554  555  556  557  558  559  580  600  601  602  603  604  605  606  607
 [31]  608  609  610  611  612  613  614  615  616  617  618  619  620  621  622
 [46]  623  624  625  626  627  628  629  630  631  632  633  634  635  636  637
 [61]  638  639  640  641  642  643  644  645  646  647  648  649  650  651  652
 [76]  653  654  655  656  657  658  659  700  701  702  703  704  705  706  707
 [91]  708  709  710  711  712  713  714  715  716  717  718  719  720  721  722
[106]  723  724  725  726  727  728  729  730  731  732  733  734  735  736  737
[121]  738  739  740  741  742  743  744  745  746  747  748  749  750  751  752
[136]  753  754  755  756  757  758  759  763  800  801  802  803  804  805  806
[151]  807  808  809  810  811  812  813  814  815  816  817  818  819  820  821
[166]  822  823  824  825  826  827  828  829  830  831  832  833  834  835  836
[181]  837  838  839  840  841  842  843  844  845  846  847  848  849  850  851
[196]  852  853  854  855  856  857  858  859  900  901  902  903  904  905  906
[211]  907  908  909  910  911  912  913  914  915  916  917  918  919  920  921
[226]  922  923  924  925  926  927  928  929  930  931  932  933  934  935  936
[241]  937  938  939  940  941  942  943  944  945  946  947  948  949  950  951
[256]  952  953  954  955  956  957  958  959 1000 1001 1002 1003 1004 1005 1006
[271] 1007 1008 1009 1010 1011 1012 1013 1014 1015 1016 1017 1018 1019 1020 1021
[286] 1022 1023 1024 1025 1026 1027 1028 1029 1030 1031 1032 1033 1034 1035 1036
[301] 1037 1038 1039 1040 1041 1042 1043 1044 1045 1046 1047 1048 1049 1050 1051
[316] 1052 1053 1054 1055 1056 1057 1058 1059 1100 1101 1102 1103 1104 1105 1107
[331] 1108 1112 1115

In [0]:
# replace_with_na function is from naniar
df <- df %>%
  replace_with_na(replace = list(survey_time_start_MDT = c(0, 1, 50, 98)))

### survey_time_end

In [24]:
sort(unique(df$survey_time_end))

[1]    1    2   85  107  507  550  553  554  555  556  557  558  600  601  602
 [16]  603  604  605  606  607  608  609  610  611  612  613  614  615  616  617
 [31]  618  619  620  621  622  623  624  625  626  627  628  629  630  631  632
 [46]  633  634  635  636  637  638  639  640  641  642  643  644  645  646  647
 [61]  648  649  650  651  652  653  654  655  656  657  658  659  700  701  702
 [76]  703  704  705  706  707  708  709  710  711  712  713  714  715  716  717
 [91]  718  719  720  721  722  723  724  725  726  727  728  729  730  731  732
[106]  733  734  735  736  737  738  739  740  741  742  743  744  745  746  747
[121]  748  749  750  751  752  753  754  755  756  757  758  759  800  801  802
[136]  803  804  805  806  807  808  809  810  811  812  813  814  815  816  817
[151]  818  819  820  821  822  823  824  825  826  827  828  829  830  831  832
[166]  833  834  835  836  837  838  839  840  841  842  843  844  845  846  847
[181]  848  849  850  851  852  853  854  855  856  857  858  859  900  901  902
[196]  903  904  905  906  907  908  909  910  911  912  913  914  915  916  917
[211]  918  919  920  921  922  923  924  925  926  927  928  929  930  931  932
[226]  933  934  935  936  937  938  939  940  941  942  943  944  945  946  947
[241]  948  949  950  951  952  953  954  955  956  957  958  959 1000 1001 1002
[256] 1003 1004 1005 1006 1007 1008 1009 1010 1011 1012 1013 1014 1015 1016 1017
[271] 1018 1019 1020 1021 1022 1023 1024 1025 1026 1027 1028 1029 1030 1031 1032
[286] 1033 1034 1035 1036 1037 1038 1039 1040 1041 1042 1043 1044 1045 1046 1047
[301] 1048 1049 1050 1051 1052 1053 1054 1055 1056 1057 1058 1059 1100 1101 1102
[316] 1103 1104 1105 1106 1107 1108 1109 1110 1111 1112 1113 1114 1115 1117 1119
[331] 1122 1125 1146 6658

In [0]:
# replace_with_na function is from naniar
df <- df %>%
  replace_with_na(replace = list(survey_time_end_MDT = c(6658, 1, 2, 85, 107)))

# Set DataTypes

In [26]:
str(df)

'data.frame':	117911 obs. of  30 variables:
 $ survey_ID                         : int  1 1 1 1 1 1 1 1 1 1 ...
 $ survey_date                       : chr  "2010-05-31" "2010-05-31" "2010-05-31" "2010-05-31" ...
 $ survey_time_start_MDT             : int  709 709 709 709 709 709 709 709 709 709 ...
 $ survey_time_end_MDT               : int  719 719 719 719 719 719 719 719 719 719 ...
 $ survey_visit                      : int  1 1 1 1 1 1 1 1 1 1 ...
 $ survey_grid_point                 : int  476 476 476 476 476 476 476 476 476 476 ...
 $ survey_observer                   : chr  "Eric Rasmussen" "Eric Rasmussen" "Eric Rasmussen" "Eric Rasmussen" ...
 $ survey_wind_code                  : int  1 1 1 1 1 1 1 1 1 1 ...
 $ survey_sky_code                   : int  2 2 2 2 2 2 2 2 2 2 ...
 $ survey_noise_code                 : int  2 2 2 2 2 2 2 2 2 2 ...
 $ survey_temperature_F              : int  50 50 50 50 50 50 50 50 50 50 ...
 $ records_ID                        : int  1 2 3 4 5 6 7 

## survey_date

In [0]:
df$survey_date <- as_date(df$survey_date)

## survey_time_*

In [28]:
df$survey_time_start_MDT <- hms::as_hms(parse_date_time(df$survey_time_start_MDT, "HM"))
df$survey_time_end_MDT <- hms::as_hms(parse_date_time(df$survey_time_end_MDT, "HM"))

Warning message:
“ 29 failed to parse.”


## records_detect_*

In [0]:
df <- df %>%
  mutate_at(vars(16:18), as.logical)

## records_location_code

In [30]:
fct_count(df$records_location_code)

f,n
<fct>,<int>
0,11959
1,18433
10,141
11,1452
12,860
13,783
14,256
15,5448
16,8384


In [31]:
str(df)

'data.frame':	117911 obs. of  30 variables:
 $ survey_ID                         : int  1 1 1 1 1 1 1 1 1 1 ...
 $ survey_date                       : Date, format: "2010-05-31" "2010-05-31" ...
 $ survey_time_start_MDT             : 'hms' num  07:09:00 07:09:00 07:09:00 07:09:00 ...
  ..- attr(*, "units")= chr "secs"
 $ survey_time_end_MDT               : 'hms' num  07:19:00 07:19:00 07:19:00 07:19:00 ...
  ..- attr(*, "units")= chr "secs"
 $ survey_visit                      : int  1 1 1 1 1 1 1 1 1 1 ...
 $ survey_grid_point                 : int  476 476 476 476 476 476 476 476 476 476 ...
 $ survey_observer                   : chr  "Eric Rasmussen" "Eric Rasmussen" "Eric Rasmussen" "Eric Rasmussen" ...
 $ survey_wind_code                  : int  1 1 1 1 1 1 1 1 1 1 ...
 $ survey_sky_code                   : int  2 2 2 2 2 2 2 2 2 2 ...
 $ survey_noise_code                 : int  2 2 2 2 2 2 2 2 2 2 ...
 $ survey_temperature_F              : int  50 50 50 50 50 50 50 50 50 50 ...
 

In [34]:
fct_count(as.factor(df$survey_wind_code))

f,n
<fct>,<int>
0,26461
1,48833
2,32971
3,7891
4,1472
5,155
NA,128


In [44]:
df %>%
  filter(is.na(survey_wind_code))

survey_ID,survey_date,survey_time_start_MDT,survey_time_end_MDT,survey_visit,survey_grid_point,survey_observer,survey_wind_code,survey_sky_code,survey_noise_code,⋯,records_interval,records_location_code,habitat_location_description,function_spe_name_common,function_habitat_preference,function_food_preference,function_nesting_behavior,function_feeding_behavior,function_taxa_order,function_taxa_family
<int>,<date>,<time>,<time>,<int>,<int>,<chr>,<int>,<int>,<int>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1397,2011-05-31,08:45:00,08:55:00,NA,355,Katharine Stone,NA,NA,NA,⋯,1,7,Draw (Populus canopy),American Kestrel,Open Woodland,Insects,Cavity,Soaring,Falconiformes,Falconidae
1397,2011-05-31,08:45:00,08:55:00,NA,355,Katharine Stone,NA,NA,NA,⋯,1,7,Draw (Populus canopy),American Kestrel,Open Woodland,Insects,Cavity,Soaring,Falconiformes,Falconidae
1397,2011-05-31,08:45:00,08:55:00,NA,355,Katharine Stone,NA,NA,NA,⋯,1,7,Draw (Populus canopy),Mourning Dove,Open Woodland,Seeds,Tree,Ground Forager,Columbiformes,Columbidae
1397,2011-05-31,08:45:00,08:55:00,NA,355,Katharine Stone,NA,NA,NA,⋯,1,1,Open grassland,Bank Swallow,Lake/Pond,Insects,Burrow,Aerial Forager,Passeriformes,Hirundinidae
1397,2011-05-31,08:45:00,08:55:00,NA,355,Katharine Stone,NA,NA,NA,⋯,1,1,Open grassland,Cliff Swallow,Lake/Pond,Insects,Cliff,Aerial Forager,Passeriformes,Hirundinidae
1397,2011-05-31,08:45:00,08:55:00,NA,355,Katharine Stone,NA,NA,NA,⋯,1,1,Open grassland,Western Meadowlark,Grassland,Insects,Ground,Ground Forager,Passeriformes,Icteridae
1397,2011-05-31,08:45:00,08:55:00,NA,355,Katharine Stone,NA,NA,NA,⋯,1,1,Open grassland,Western Meadowlark,Grassland,Insects,Ground,Ground Forager,Passeriformes,Icteridae
1397,2011-05-31,08:45:00,08:55:00,NA,355,Katharine Stone,NA,NA,NA,⋯,1,1,Open grassland,Western Meadowlark,Grassland,Insects,Ground,Ground Forager,Passeriformes,Icteridae
1397,2011-05-31,08:45:00,08:55:00,NA,355,Katharine Stone,NA,NA,NA,⋯,1,1,Open grassland,Vesper Sparrow,Grassland,Insects,Ground,Ground Forager,Passeriformes,Emberizidae


# Output

In [0]:
# last output 2020-05-19 by esamsoe
# output = 'bird_surveyInfo_records_function.csv'
# write_csv(df, path = output)

# Notes

Attempting to upload this CSV into BigQuery results in 
* "Error executing job" 
* "Error while reading data, error message: CSV table encountered too many errors, giving up. Rows: 12912; errors: 1. Please look into the errors[] collection for more details."
* "Error while reading data, error message: Could not parse 'NA' as INT64 for field survey_wind_code (position 7) starting at location 2652793"

To parse the CSV correctly refer to [bq load syntax](https://docs.google.com/document/d/1nCl_sbl2Eac9Qb3bZymdRJl3voXC4lPUqBeyLZRlwow/edit?usp=sharing)